In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import glob
from tqdm import tqdm
warnings.filterwarnings("ignore")

2025-11-19 01:10:55.137461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763514655.162171     251 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763514655.169278     251 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
train = pd.read_parquet("/kaggle/input/train_dataset.parquet")
test = pd.read_parquet("/kaggle/input/test_dataset.parquet")
train['target'] = np.log(train['price_TARGET'])

In [3]:
train['target']

0        10.839581
1        12.180755
2        15.796650
3        13.880362
4        10.896739
           ...    
69995    13.691080
69996    13.208541
69997    12.581079
69998    12.587928
69999    13.921671
Name: target, Length: 70000, dtype: float64

In [4]:
len(os.listdir("/kaggle/input/AvitoAuto/АвтоПрайс/test_images"))

98075

In [6]:
len(os.listdir("/kaggle/input/AvitoAuto/АвтоПрайс/train_images"))

273873

# EMBEDDINGS

In [ ]:
processor = AutoImageProcessor.from_pretrained("WinKawaks/vit-tiny-patch16-224", use_fast=True)
vit_model = AutoModel.from_pretrained("WinKawaks/vit-tiny-patch16-224")
del vit_model.pooler
vit_model.pooler = torch.nn.Identity()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model.to(device)
vit_model.eval()

In [ ]:
train_image_dir = '/kaggle/input/AvitoAuto/АвтоПрайс/train_images'
test_image_dir = '/kaggle/input/AvitoAuto/АвтоПрайс/test_images'

print(f"Train images dir exists: {os.path.exists(train_image_dir)}")
print(f"Test images dir exists: {os.path.exists(test_image_dir)}")

# Проверим какие файлы есть в директориях
train_files = glob.glob(os.path.join(train_image_dir, "*.jpg"))
test_files = glob.glob(os.path.join(test_image_dir, "*.jpg"))
print(f"Found {len(train_files)} train images, {len(test_files)} test images")

In [ ]:
class CarImageDataset(Dataset):
    def __init__(self, folder, train=True):
        self.folder = folder
        self.image_paths = []
        self.ID2Images = {}
        self.train = train

        for img in sorted(os.listdir(folder)):
            path = os.path.join(folder, img)
            ID = img.split("_")[0]
            self.image_paths.append(path)
            self.ID2Images.setdefault(int(ID), [])
            self.ID2Images[int(ID)].append(path)

        self.ids = sorted(self.ID2Images.keys())
    
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        ID = self.ids[idx]
        image_paths = self.ID2Images[ID]
        
        images = [Image.open(img).convert("RGB") for img in image_paths]
        processed = processor(images, return_tensors="pt")
        processed = {k: v.to(device) for k, v in processed.items()}

        with torch.inference_mode():
            outputs = vit_model(**processed)
            cls_embed = outputs.last_hidden_state[:, 0, :]

        if self.train:
            target = train.loc[train[train.ID == ID].index[0], "price_TARGET"]
            return ID, torch.mean(cls_embed, dim=0)
        else:
            return ID, torch.mean(cls_embed, dim=0)

In [ ]:
dataset = CarImageDataset(train_image_dir, train=True)
data_test = CarImageDataset(test_image_dir, train=False)
train_loader = DataLoader(dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(data_test, batch_size=32, shuffle=False)


In [ ]:
mapper = {}

for batch in tqdm(train_loader):
    X, y = batch
    for i in range(len(X)):
        ind = int(X[i])
        val = y[i].detach().cpu().numpy()
        mapper[ind] = val

In [ ]:
emb_cols = [f"emb_{i}" for i in range(192)]
vectors = train['ID'].map(mapper).tolist()
vectors_df = pd.DataFrame(vectors, columns=emb_cols, index=train.index)
train[emb_cols] = vectors_df

# Finetune

In [7]:
processor = AutoImageProcessor.from_pretrained("WinKawaks/vit-tiny-patch16-224", use_fast=True)
vit_model = AutoModel.from_pretrained("WinKawaks/vit-tiny-patch16-224")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model.to(device)
vit_model.eval()

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=192, out_features=192, bias=True)
            (key): Linear(in_features=192, out_features=192, bias=True)
            (value): Linear(in_features=192, out_features=192, bias=True)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=192, out_features=192, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=192, out_features=768, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): ViTOutput(
          (de

In [8]:
class CarImageDataset(Dataset):
    def __init__(self, folder, df, processor, train=True):
        self.folder = folder
        self.train = train
        self.processor = processor
        self.samples = [] 

        # Создаем словарь {ID: target} для быстрого поиска
        id_to_target = {}
        if self.train and df is not None:
            id_to_target = dict(zip(df['ID'], df['price_TARGET']))

        # Сканируем папку
        valid_extensions = ('.jpg', '.jpeg', '.png')
        for img in sorted(os.listdir(folder)):
            if not img.lower().endswith(valid_extensions):
                continue

            path = os.path.join(folder, img)
            try:
                # Предполагаем формат имени файла "ID_номер.jpg"
                file_id = int(img.split("_")[0])
            except ValueError:
                continue

            if self.train:
                # Добавляем только если ID есть в train датафрейме
                if file_id in id_to_target:
                    target = id_to_target[file_id]
                    self.samples.append((path, target))
            else:
                self.samples.append(path)
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        if self.train:
            path, target = self.samples[idx]
        else:
            path = self.samples[idx]
            target = 0.0 # Заглушка
        
        image = Image.open(path).convert("RGB")
        
        # Превращаем картинку в тензор
        processed = self.processor(image, return_tensors="pt")
        # Убираем лишнюю размерность батча [1, 3, 224, 224] -> [3, 224, 224]
        pixel_values = processed['pixel_values'][0]

        if self.train:
            return pixel_values, torch.tensor(target, dtype=torch.float32)
        else:
            return pixel_values

In [9]:
class ViTForRegression(nn.Module):
    def __init__(self, base_model):
        super(ViTForRegression, self).__init__()
        self.base_model = base_model
        
        # Размер скрытого слоя у tiny vit обычно 192
        hidden_size = base_model.config.hidden_size 
        
        # Линейный слой: превращает вектор 192 в 1 число (цену)
        self.regressor = nn.Linear(hidden_size, 1) 

    def forward(self, pixel_values):
        # Прогоняем через ViT
        outputs = self.base_model(pixel_values=pixel_values)
        
        # Берем last_hidden_state: [Batch, Seq_Len, Hidden]
        # Нам нужен только CLS токен (индекс 0 по Seq_Len)
        cls_token = outputs.last_hidden_state[:, 0, :] 
        
        # Прогоняем через регрессор
        prediction = self.regressor(cls_token)
        
        # Возвращаем [Batch], убираем лишнюю размерность [Batch, 1] -> [Batch]
        return prediction.squeeze(-1)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor, AutoModel
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# --- 0. Подготовка данных (Split) ---
# Делим датафрейм train на обучающую (85%) и валидационную (15%) выборки
# train_df используется для обучения, val_df - для проверки
train_df, val_df = train_test_split(train, test_size=0.15, random_state=42)

print(f"Train samples: {len(train_df)}, Validation samples: {len(val_df)}")

# --- 1. Инициализация ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = AutoImageProcessor.from_pretrained("WinKawaks/vit-tiny-patch16-224", use_fast=True)
base_vit = AutoModel.from_pretrained("WinKawaks/vit-tiny-patch16-224")

model = ViTForRegression(base_vit)
model.to(device)

# --- 2. Создание DataLoader'ов ---
# Для валидации тоже ставим train=True, так как нам нужны таргеты для подсчета ошибки
train_dataset = CarImageDataset(folder="/kaggle/input/AvitoAuto/АвтоПрайс/train_images", df=train_df, processor=processor, train=True)
val_dataset = CarImageDataset(folder="/kaggle/input/AvitoAuto/АвтоПрайс/train_images", df=val_df, processor=processor, train=True)

batch_size = 32 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# --- 3. Настройки обучения ---
criterion = nn.MSELoss()
# L1 Loss нужен только для метрики (чтобы видеть ошибку в рублях/долларах), оптимизируем всё равно MSE
metric_l1 = nn.L1Loss() 

optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Scheduler: Плавно снижает LR от 5e-5 до 0 в течение всех эпох
num_epochs = 10 # Увеличь количество эпох, 1 маловато для шедулера
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# --- 4. Цикл обучения ---
best_val_loss = float('inf') # Для сохранения лучшей модели

for epoch in range(num_epochs):
    # === TRAIN LOOP ===
    model.train()
    train_loss = 0.0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
    
    for images, targets in progress_bar:
        images = images.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})
    
    # Шаг шедулера в конце эпохи
    scheduler.step()
    
    avg_train_loss = train_loss / len(train_loader)

    # === VALIDATION LOOP ===
    model.eval() # Выключаем Dropout и Batchnorm update
    val_loss_mse = 0.0
    val_loss_mae = 0.0
    
    # Используем inference_mode для ускорения и экономии памяти
    with torch.inference_mode():
        for images, targets in val_loader:
            images = images.to(device)
            targets = targets.to(device)
            
            outputs = model(images)
            
            # Считаем MSE для Лосса
            loss = criterion(outputs, targets)
            val_loss_mse += loss.item()
            
            # Считаем L1 (MAE) для понимания ошибки в абсолютных величинах
            mae = metric_l1(outputs, targets)
            val_loss_mae += mae.item()

    avg_val_mse = val_loss_mse / len(val_loader)
    avg_val_mae = val_loss_mae / len(val_loader)
    
    print(f"Epoch {epoch+1}: Train MSE: {avg_train_loss:.4f} | Val MSE: {avg_val_mse:.4f} | Val MAE: {avg_val_mae:.2f}")
    print(f"Current LR: {optimizer.param_groups[0]['lr']:.2e}")

    # Сохраняем лучшую модель (по MSE)
    if avg_val_mse < best_val_loss:
        print(f"Validation loss improved ({best_val_loss:.4f} -> {avg_val_mse:.4f}). Saving model...")
        best_val_loss = avg_val_mse
        torch.save(model.state_dict(), "best_vit_price_model.pth")

print("Training complete.")

Train samples: 59500, Validation samples: 10500


Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/10 [Train]:   2%|▏         | 111/7276 [00:13<13:39,  8.75it/s, loss=1.6e+12] 

In [ ]:
model.load_state_dict(state_dict)

# PREDICT

In [ ]:
cat_features = ['equipment', 'body_type', 'drive_type', 'engine_type', 'doors_number', 'color', 'pts', 'diski', 'audiosistema', 'electropodemniki', 'fary', 'salon', 'upravlenie_klimatom',
                'usilitel_rul', 'steering_wheel', 'crashes_count', 'owners_count']
numeric_features = ['mileage', 'latitude', 'longitude'] + np.array(emb_cols).tolist()

listed_features = ['aktivnaya_bezopasnost_mult', 'audiosistema_mult', 'shini_i_diski_mult', 'electroprivod_mult', 'fary_mult', 'multimedia_navigacia_mult', 'obogrev_mult', 'pamyat_nastroek_mult', 'podushki_bezopasnosti_mult',
                   'pomosh_pri_vozhdenii_mult', 'protivoygonnaya_sistema_mult', 'salon_mult', 'upravlenie_klimatom_mult']

target = ['price_TARGET']
ID = ['ID']

In [ ]:
from tqdm import tqdm

In [ ]:
import pandas as pd

# Собираем все уникальные метки для train и test
labels = {col: set().union(*train[col]) for col in listed_features}
test_labels = {col: set().union(*test[col]) for col in listed_features}

# One-hot для train
for col in listed_features:
    # Разворачиваем список в строки
    exploded = train[[col]].explode(col)
    # Получаем one-hot кодировку
    dummies = pd.get_dummies(exploded[col], prefix=col)
    # Складываем обратно по индексам
    dummies = dummies.groupby(exploded.index).max()
    # Добавляем в train
    train = train.join(dummies)

# One-hot для test
for col in listed_features:
    exploded = test[[col]].explode(col)
    dummies = pd.get_dummies(exploded[col], prefix=col)
    dummies = dummies.groupby(exploded.index).max()
    test = test.join(dummies)

In [ ]:
for i, lbs in enumerate(labels):
    print(lbs, labels[lbs] == test_labels[lbs])

In [ ]:
for col in test.columns:
    if col not in target and col not in listed_features and col not in numeric_features and col not in cat_features and col not in ID:
        cat_features.append(col)

## CATBOOST

In [ ]:
!pip install -U scikit-learn

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
!pip install -U "scikit-learn<1.5"

In [ ]:
!pip install -U catboost

In [ ]:
from catboost import CatBoostRegressor
import numpy as np

test[cat_features] = test[cat_features].fillna("NaN")
train[cat_features] = train[cat_features].fillna("NaN")

train['target'] = np.log(train['price_TARGET'])

class MedianAPE:
    
    @staticmethod
    def get_median_ape(y_true, y_pred, eps=1e-8):
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        return np.median(np.abs(y_pred - y_true) / np.clip(np.abs(y_true), eps, None))
    
    def is_max_optimal(self):
        return False  # чем меньше MdAPE, тем лучше
    
    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        y_true = np.array(target)
        y_pred = np.array(approxes[0])
        score = self.get_median_ape(y_true, y_pred)
        return score, 1  # (значение метрики, вес)
    
    def get_final_error(self, error, weight):
        return error


X = train[cat_features + numeric_features]
y = train['target']

indices = np.arange(len(train))
np.random.seed(52)
np.random.shuffle(indices)

split = int(0.9 * len(indices))
train_idx, val_idx = indices[:split], indices[split:]

X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=52, shuffle=True)

model = CatBoostRegressor(
    iterations=2500,
    learning_rate=0.03,
    depth=10,
    cat_features=cat_features,
    loss_function="RMSE",
    eval_metric="RMSE",
    verbose=100,
    random_seed=42,
    task_type="GPU"
)

model.fit(X=X_train, y=y_train, eval_set=(X_val, y_val),
          cat_features=cat_features, use_best_model=True)


def median_APE(y_true, y_pred, eps=1e-8):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.median(np.abs(y_pred - y_true) / np.clip(np.abs(y_true), eps, None))

y_pred = np.exp(model.predict(X_val))
y_val = train.loc[X_val.index, "price_TARGET"]
score = median_APE(y_val, y_pred)
print(f"Median_APE: {score}\n" +
      f"LB: {1 / (1 + score)}\n" +
      f"MAE: {mean_absolute_error(y_val, y_pred)}\n" +
      f"Mean_APE: {mean_absolute_percentage_error(y_val, y_pred)}")

In [ ]:
submission = pd.DataFrame({"ID": test["ID"],
              "target": np.exp(model.predict(test[cat_features + numeric_features]))})

In [ ]:
submission.to_csv("baseline.csv", index=False)

In [ ]:
submission

# OPTUNA

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import numpy as np
import optuna
from optuna.samplers import RandomSampler

# Предобработка данных
test[cat_features] = test[cat_features].fillna("NaN")
train[cat_features] = train[cat_features].fillna("NaN")
train['target'] = np.log(train['price_TARGET'])

# Метрика MedianAPE
class MedianAPE:
    @staticmethod
    def get_median_ape(y_true, y_pred, eps=1e-8):
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        return np.median(np.abs(y_pred - y_true) / np.clip(np.abs(y_true), eps, None))
    
    def is_max_optimal(self):
        return False
    
    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        y_true = np.array(target)
        y_pred = np.array(approxes[0])
        score = self.get_median_ape(y_true, y_pred)
        return score, 1
    
    def get_final_error(self, error, weight):
        return error

def median_APE(y_true, y_pred, eps=1e-8):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.median(np.abs(y_pred - y_true) / np.clip(np.abs(y_true), eps, None))

# Разделение данных
X = train[cat_features + numeric_features]
y = train['target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=52, shuffle=True)

In [ ]:
# Функция для оптимизации
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 1000, 7500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'depth': trial.suggest_int('depth', 4, 11),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_strength': trial.suggest_float('random_strength', 0.1, 2),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 250),
        'loss_function': 'RMSE',
        'eval_metric': MedianAPE(),
        'cat_features': cat_features,
        'verbose': False,
        'random_seed': 42,
        'task_type': 'GPU',
        # "early_stopping_rounds": 250
    }
    
    model = CatBoostRegressor(**params)
    
    try:
        model.fit(
            X=X_train, y=y_train,
            eval_set=(X_val, y_val),
            early_stopping_rounds=100,
            verbose=False
        )
        
        # Предсказание и оценка
        y_pred = np.exp(model.predict(X_val))
        y_true = train.loc[X_val.index, "price_TARGET"]
        score = median_APE(y_true, y_pred)
        
    except Exception as e:
        print(f"Ошибка при обучении: {e}")
        score = float('inf')
        
    return score

# Исследование
study = optuna.create_study(
    direction='minimize',
    sampler=RandomSampler(seed=16)
)
study.optimize(objective, n_trials=600)

# Лучшие параметры
print("Лучшие параметры:")
for key, value in study.best_params.items():
    print(f"{key}: {value}")
print(f"Лучший Median_APE: {study.best_value}")

In [ ]:
study.best_params

In [ ]:
pars = {'iterations': 4773,
 'learning_rate': 0.023540714865270384,
 'depth': 11,
 'l2_leaf_reg': 4.339528598041656,
 'random_strength': 1.4767698885793084,
 'bagging_temperature': 0.5228168517505888,
 'border_count': 186,
 'grow_policy': 'Depthwise',
 'min_data_in_leaf': 76}

In [ ]:
pars_redacted = {'iterations': 1242,
 'learning_rate': 0.023540714865270384,
 'depth': 11,
 'l2_leaf_reg': 4.339528598041656,
 'random_strength': 1.4767698885793084,
 'bagging_temperature': 0.5228168517505888,
 'border_count': 186,
 'grow_policy': 'Depthwise',
 'min_data_in_leaf': 76}

In [ ]:
# Финальное обучение с лучшими параметрами
best_model = CatBoostRegressor(
    **pars,
    cat_features=cat_features,
    loss_function="RMSE",
    eval_metric=MedianAPE(),
    verbose=100,
    random_seed=42,
    task_type="GPU"
)

best_model.fit(
    X=X_train, y=y_train,
    eval_set=(X_val, y_val),
    use_best_model=True,
    early_stopping_rounds=250
)

In [ ]:
# Финальная оценка
y_pred = np.exp(best_model.predict(X_val))
y_true = train.loc[X_val.index, "price_TARGET"]
final_score = median_APE(y_true, y_pred)

print(f"\nФинальный Median_APE: {final_score}")
print(f"LB score: {1 / (1 + final_score)}")

In [ ]:
submission = pd.DataFrame({"ID": test["ID"],
              "target": np.exp(best_model.predict(test[cat_features + numeric_features]))})

In [ ]:
submission.to_csv("opt.csv", index=False)

In [ ]:
submission